## Data Preprocessing for food Mama project: 

In [1]:
import pandas as pd
import os

### Loading the Datasets into Pandas dataframe

In [2]:
path = "datasets/"
files = os.listdir(path)

dfs = []
for file in files:
    filename = file.split("_")
    dfs.append(filename[0])
    exec(dfs[-1] + '= pd.read_csv(path + file)')

print("Name of the dataframes created: ", *dfs, sep="\n")

Name of the dataframes created: 
food
item
recipe


### 1) *foods* table: 

In [3]:
#exec("print(" + dfs[0] + ".head())")
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564 entries, 0 to 563
Data columns (total 7 columns):
id              564 non-null int64
name            564 non-null object
created_at      564 non-null object
updated_at      564 non-null object
availability    563 non-null object
category_id     563 non-null float64
ancestry        159 non-null object
dtypes: float64(1), int64(1), object(5)
memory usage: 30.9+ KB


- Create a copy of table and then drop unnecessary columns, replace "id" with "food_id", and try to fill the missing data. 

In [4]:
food_copy = food.copy()
food.drop(["created_at","updated_at"], axis=1, inplace=True)
food.rename(columns={'id': 'food_id'}, inplace=True)
food.sort_index().head()

,food_id,name,availability,category_id,ancestry
0,7,oignon,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,NaN
1,4,semoule,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",21.0,NaN
2,42,mâche,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,86
3,14,roquette,"05, 06, 07, 08, 09, 10, 11",14.0,86
4,54,crème liquide,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",32.0,NaN


In [5]:
# availability column missing data
food[food.availability.isnull()]

,food_id,name,availability,category_id,ancestry
64,53,pomme de terre,NaN,14.0,NaN


In [6]:
# Check the children's availability data 
ancestry_id = food[food.availability.isnull()].food_id
food[food.ancestry == str(ancestry_id.values[0])]

,food_id,name,availability,category_id,ancestry
122,180,purée de pommes de terre,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",47.0,53
181,243,pomme de terre nouvelle,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,53
466,274,pomme de terre rouge,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,53
475,157,pommes de terre rissolées,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",47.0,53
519,521,chips,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",19.0,53


In [7]:
# Fill availability cell with one of its children data (here with the 1st one)
child_indexs = food[food.ancestry == str(ancestry_id.values[0])].index
ancestry_index = food[food.availability.isnull()].index
food.loc[ancestry_index[0], ["availability"]] = food.loc[child_indexs[0], ["availability"]]
food.loc[ancestry_index[0]]

food_id                                                     53
name                                            pomme de terre
availability    01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12
category_id                                                 14
ancestry                                                   NaN
Name: 64, dtype: object

In [8]:
# category_id column missing data
food[food.category_id.isnull()]

,food_id,name,availability,category_id,ancestry
194,288,couscous,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",NaN,NaN


In [9]:
# Check similar food using the same name 
food[food.name.apply(lambda x: "couscous" in x)]

,food_id,name,availability,category_id,ancestry
194,288,couscous,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",NaN,NaN


In [10]:
# Check for similar Moroccan food such as "semoule" 
food[food.name.apply(lambda x: "semoule" in x)]

,food_id,name,availability,category_id,ancestry
1,4,semoule,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",21.0,NaN
563,597,semoule de maïs,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",20.0,NaN


In [11]:
# Fill the category_id with the value of "semoule"
index_num = food[food.category_id.isnull()].index
food.loc[index_num[0], ["category_id"]] = food[food.name == "semoule"]["category_id"].values[0]

- Check different value of *ancestry* column and try to minimize the "NaN" values.

In [12]:
food.ancestry.value_counts(dropna=False)

NaN        405
3           14
5           12
509         10
327          9
96           8
508          8
86           8
510          8
237          7
21           6
317          5
510/404      5
53           5
308          5
511          4
349          4
7            3
516          3
519          3
269          3
510/512      2
194          2
36           2
334          2
51           2
84           2
198          2
487          2
247          1
409          1
19           1
259          1
506          1
174          1
152          1
43           1
508/277      1
503          1
262          1
66           1
508/422      1
Name: ancestry, dtype: int64

In [13]:
food[food.ancestry == "510/404"]

,food_id,name,availability,category_id,ancestry
184,187,saucisse fumée,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/404
334,402,saucisse de Strasbourg,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/404
393,407,chipolata,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/404
396,200,saucisse de Francfort,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/404
506,186,saucisse de porc,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/404


In [14]:
# Verify both ancestries 
food[(food.food_id == 510) | (food.food_id == 404)]

,food_id,name,availability,category_id,ancestry
489,510,porc,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",27.0,NaN
507,404,saucisse,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510


In [15]:
# Change the ancestry value with the subcategory id number
sub_cat = food[food.ancestry == "510/404"]["ancestry"]
food.loc[sub_cat.index, "ancestry"] = sub_cat.values[0].split("/")[1]

- Repeat the same procedure for other values 

In [16]:
food[food.ancestry == "510/512"]

,food_id,name,availability,category_id,ancestry
189,211,saucisse sèche,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/512
321,361,salami,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510/512


In [17]:
food[(food.food_id == 510) | (food.food_id == 512)]

,food_id,name,availability,category_id,ancestry
489,510,porc,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",27.0,NaN
508,512,saucisson,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",23.0,510


In [18]:
food.loc[[189,321], "ancestry"] = "512"

In [19]:
food[(food.ancestry == "508/277") | (food.ancestry == "508/422")]

,food_id,name,availability,category_id,ancestry
329,560,chou frisé,"01, 02, 03, 10, 11, 12",14.0,508/277
521,526,chou vert,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,508/422


In [20]:
food[(food.food_id == 508) | (food.food_id == 277) | (food.food_id == 422)]

,food_id,name,availability,category_id,ancestry
326,422,chou pommé,"01, 02, 03, 04, 10, 11, 12",14.0,508
392,277,chou kale,"01, 02, 03, 10, 11, 12",14.0,508
503,508,chou,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,NaN


In [21]:
food[(food.ancestry == "508") | (food.ancestry == "277") | (food.ancestry == "422")]

,food_id,name,availability,category_id,ancestry
124,156,chou-fleur,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,508
195,315,chou chinois,"01, 02, 05, 06, 07, 08, 09, 10, 11, 12",14.0,508
196,287,chou blanc,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,508
326,422,chou pommé,"01, 02, 03, 04, 10, 11, 12",14.0,508
392,277,chou kale,"01, 02, 03, 10, 11, 12",14.0,508
469,420,chou rouge,"01, 02, 03, 04, 05, 06, 07, 08, 09, 10, 11, 12",14.0,508
484,419,chou de Bruxelles,"01, 02, 03, 09, 10, 11, 12",14.0,508
530,421,chou romanesco,"06, 07, 08, 09",14.0,508


In [22]:
food.loc[[329,521], "ancestry"] = "508"

- Change the "NaN" value in *ancestry* column with "1" (to be considered as a root position) for the foods line with at least one child in the table. And those that don't have any (parent/child) relation with "0".   

In [23]:
id_list = food[food.ancestry.isnull()].index
for i in id_list:
    if len(food[food.ancestry == str(food.loc[i, "food_id"])]) > 0:
        food.loc[i, "ancestry"] = "1"
    else:
        food.loc[i, "ancestry"] = "0"

food.ancestry.value_counts(normalize=True, dropna=False)

0      0.650709
1      0.067376
3      0.024823
5      0.021277
509    0.017730
508    0.017730
327    0.015957
86     0.014184
510    0.014184
96     0.014184
237    0.012411
21     0.010638
308    0.008865
404    0.008865
317    0.008865
53     0.008865
511    0.007092
349    0.007092
269    0.005319
516    0.005319
519    0.005319
7      0.005319
334    0.003546
51     0.003546
194    0.003546
512    0.003546
487    0.003546
198    0.003546
84     0.003546
36     0.003546
259    0.001773
19     0.001773
174    0.001773
152    0.001773
503    0.001773
66     0.001773
262    0.001773
506    0.001773
43     0.001773
409    0.001773
247    0.001773
Name: ancestry, dtype: float64

### 2) *recipes* table: 

In [24]:
recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 12 columns):
id               1234 non-null int64
title            1234 non-null object
servings         1234 non-null int64
ingredients      1234 non-null object
instructions     1234 non-null object
created_at       1234 non-null object
updated_at       1234 non-null object
recommendable    23 non-null object
status           1234 non-null object
origin           1234 non-null object
link             753 non-null object
rating           1214 non-null object
dtypes: int64(2), object(10)
memory usage: 115.8+ KB


- Create a copy of table and then drop unnecessary columns, replace "id" with "recipe_id", and try to fill the missing data.

In [25]:
recipe_copy = recipe.copy()
recipe.drop(["recommendable","status"], axis=1, inplace=True)
recipe.rename(columns={'id': 'recipe_id'}, inplace=True)
recipe.head()

,recipe_id,title,servings,ingredients,instructions,created_at,updated_at,origin,link,rating
0,9,"Salade mâche, jambon de Bayonne, mozzarella",4,4 bonne poignée de mâche \r\n40 tomate cerise ...,"Après avoir coupé en dés la mozzarella, couper...",2017-12-14 14:56:37.166524,2018-10-05 09:16:42.390163,www.marmiton.org,http://www.marmiton.org/recettes/recette_salad...,limit
1,10,Ciabattina al pesto,1,Ciabattina\r\nJambon de Parme\r\nTomates confi...,Ouvrir le pain Ciabattina en deux et tartiner ...,2017-12-14 14:56:46.270433,2018-10-05 09:16:42.45289,www.club-sandwich.net,http://www.club-sandwich.net/mobile/fiche.php?...,limit
2,402,Tortilla aux champignons et salade,4,250 g de champignons de Paris\r\r\n4 oeufs\r\r...,Préchauffer le four à 180 °C (th. 6).\r\nCoupe...,2017-12-15 16:41:37.056079,2018-10-05 09:16:42.505825,www.mangerbouger.fr,http://www.mangerbouger.fr/Manger-Mieux/Recett...,limit
3,2,Steak haché et pâtes,1,2.0 filets huile d'olive\r\n1.0 steak haché\...,"Dans une casserole, portez à ébullition un gra...",2017-12-13 16:17:24.125137,2018-10-05 09:16:42.55582,www.wecook.fr,https://www.wecook.fr/recette/steak-hache-et-p...,good
4,4,Beef Bagel,1,Pain Bagel\r\nCarpaccio de boeuf\r\nFromage fr...,Placer les tranches de carpaccio dans un plat ...,2017-12-13 17:13:29.069001,2018-10-05 09:16:42.597349,www.club-sandwich.net,http://www.club-sandwich.net/mobile/fiche.php?...,limit


In [26]:
# Check the "origin" column values for the null data in "link" column
origin_list = recipe[recipe.link.isnull()]["origin"]
origin_list.value_counts()

mama    481
Name: origin, dtype: int64

In [27]:
# Add the Mama web link recipes for the missing values in the "link" column
foodmama_path = "https://www.foodmama.fr/recipes/"
index_list = recipe[recipe.link.isnull()].index
for i in index_list:
    recipe.loc[i, ["link"]] = foodmama_path + str(recipe.loc[i, ["recipe_id"]].values[0])

# Check the first 10 lines
recipe.loc[index_list[:10], ["recipe_id", "link"]]

,recipe_id,link
28,905,https://www.foodmama.fr/recipes/905
29,1063,https://www.foodmama.fr/recipes/1063
44,1103,https://www.foodmama.fr/recipes/1103
46,1118,https://www.foodmama.fr/recipes/1118
62,1099,https://www.foodmama.fr/recipes/1099
63,906,https://www.foodmama.fr/recipes/906
70,774,https://www.foodmama.fr/recipes/774
71,908,https://www.foodmama.fr/recipes/908
80,776,https://www.foodmama.fr/recipes/776
89,777,https://www.foodmama.fr/recipes/777


In [28]:
# Check different categories in "rating" column
recipe.rating.value_counts(dropna=False)

limit        506
good         487
excellent    171
avoid         50
NaN           20
Name: rating, dtype: int64

In [29]:
# Replace "NaN" values with "unknown" rating
recipe.rating.fillna("unknown", inplace=True)
recipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234 entries, 0 to 1233
Data columns (total 10 columns):
recipe_id       1234 non-null int64
title           1234 non-null object
servings        1234 non-null int64
ingredients     1234 non-null object
instructions    1234 non-null object
created_at      1234 non-null object
updated_at      1234 non-null object
origin          1234 non-null object
link            1234 non-null object
rating          1234 non-null object
dtypes: int64(2), object(8)
memory usage: 96.5+ KB


### 3) *items* table: 

In [30]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8464 entries, 0 to 8463
Data columns (total 7 columns):
Id                      8464 non-null int64
Quantity                8464 non-null object
Recipe ingredient       8464 non-null object
Name [Food]             8464 non-null object
Title [Recipe]          8464 non-null object
Ingredients [Recipe]    8464 non-null object
Origin [Recipe]         8464 non-null object
dtypes: int64(1), object(6)
memory usage: 463.0+ KB


- Create a copy of table and then drop unnecessary columns, rename the columns, and replace index with "id".

In [31]:
item_copy = item.copy()
item.drop(["Quantity"], axis=1, inplace=True)
# Names after "_" are the tables name
item.columns = ['id', 'recipe-ingredient', 'name_foods', 'title_recipes', 'ingredients_recipes', 'origin_recipes']
item.set_index("id", inplace=True)
item.head()

,recipe-ingredient,name_foods,title_recipes,ingredients_recipes,origin_recipes
id,,,,,
1,2.0 filets huile d'olive,huile d'olive,Steak haché et pâtes,2.0 filets huile d'olive\r\n1.0 steak haché\...,www.wecook.fr
2,1.0 steak haché,steak haché,Steak haché et pâtes,2.0 filets huile d'olive\r\n1.0 steak haché\...,www.wecook.fr
3,70.0 grammes pâtes,pâtes,Steak haché et pâtes,2.0 filets huile d'olive\r\n1.0 steak haché\...,www.wecook.fr
4,500 g de semoule moyenne,semoule,Taboulé ultra-facile,500 g de semoule moyenne \r\n500 g de tomate e...,www.marmiton.org
5,500 g de tomate environ (ébouillantées pour ot...,tomate,Taboulé ultra-facile,500 g de semoule moyenne \r\n500 g de tomate e...,www.marmiton.org
